## 1. Online News Exhibition
<p>Following the changes in working habits during the COVID pandemic, your local library has decided it is time to make the newspaper archives available online. They are not sure what the response will be so they are going to trial a new system on a small set of their data. </p>
<p>The library database has a table <code>articles</code> with a record of all articles published in the newspaper. However, the table will need some preparation before the library can use it in the new system. They would like to focus the initial trial on articles from 2014 and 2015 only, and for just one journalist whose ID in the database is 1754.</p>
<p>The final results should be sorted from earliest published date to the most recent published date.</p>
<p>Below are the requirements the library has given to you for the data they need. </p>
<table>
<thead>
<tr>
<th style="text-align:left;">Requirements</th>
<th style="text-align:left;">Data Type</th>
<th style="text-align:left;">Column Alias</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;">Unique ID of each article, where the first 4 digits represents the journalist ID.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">article_key</td>
</tr>
<tr>
<td style="text-align:left;">Category of the article, in uppercase.  'Food &amp; Drink' and 'Wellness' should be collapsed  into a 'Lifestyle' category.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">category</td>
</tr>
<tr>
<td style="text-align:left;">Headline of the article, without the subtitle that appears after the colon (:).  For example, 'It Could Happen To You: A Story of SEC Overreach' becomes 'It Could Happen To You'.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">headline</td>
</tr>
<tr>
<td style="text-align:left;">Subtitles of the article, without the headline that appears before the colon (:). For example, 'It Could Happen to You: A Story of SEC Overreach' becomes 'A Story of SEC Overreach'. In cases where a headline does not have a subtitle, the missing value should read ‘None’.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">subtitle</td>
</tr>
<tr>
<td style="text-align:left;">Description of the article. The description should be reduced to the first sentence, up to and including the first period (.) from the short_description column.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">description</td>
</tr>
<tr>
<td style="text-align:left;">Keywords of the article. Multiple keywords will be seperated with a dash (-). If the keyword is missing, the value should read ‘Unknown’.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">keywords</td>
</tr>
<tr>
<td style="text-align:left;">Date the article was published. The data should be displayed in the format resembling the following: ‘Aug 13, 1995’.</td>
<td style="text-align:left;">TEXT</td>
<td style="text-align:left;">publish_date</td>
</tr>
</tbody>
</table>

In [40]:
%%sql
postgresql:///articles
    
ALTER TABLE articles 
ADD COLUMN date_n date;

update articles
SET date_n = TO_DATE(release_date, 'YY/MM/DD');



(psycopg2.errors.DuplicateColumn) column "date_n" of relation "articles" already exists

[SQL: ALTER TABLE articles 
ADD COLUMN date_n date;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [41]:
%%sql
postgresql:///articles
    
    SELECT article_key as article_key,
          UPPER(
          CASE 
             WHEN category = 'FOOD & DRINK' THEN 'Lifestyle'
             WHEN category = 'WELLNESS' THEN 'Lifestyle'
             ELSE category END ) as category,
            SPLIT_PART(headline, ':', 1) as headline,
            CASE 
                WHEN headline NOT LIKE '%:%' THEN 'None'
                ELSE SPLIT_PART(headline, ':', 2) END as subtitle,
            SPLIT_PART(short_description, '.', 1) ||'.' as description,
            CASE
                WHEN keywords IS NULL  THEN 'Unknown'
                ELSE keywords END as keywords,
            to_char(date_n, 'Mon DD, YYYY') as publish_date
                
    FROM articles
    WHERE date_part('year', date_n) IN (2014, 2015) AND article_key LIKE '1754%'
    ORDER BY date_n

188 rows affected.


article_key,category,headline,subtitle,description,keywords,publish_date
1754b3c4-21f8-11ec-bdf6-acde48001122,LIFESTYLE,21 Weird Foods You'll Serve In 2014,None,Your dinner plate is going global in 2014.,Unknown,"Jan 03, 2014"
17545488-21f8-11ec-bdf6-acde48001122,PARENTING,Life-Giving Parenting,None,I intend to converse with my children in a life-supporting fashion.,lifegiving-parenting,"Jan 06, 2014"
17549baa-21f8-11ec-bdf6-acde48001122,PARENTING,Danielle Jonas,Medication Is Helping Me With Anxiety During My Pregnancy,Watch the full segment with Danielle Jonas on HuffPost Live.,danielle-jonas-anxiety-medication-pregnancy,"Jan 08, 2014"
175459ec-21f8-11ec-bdf6-acde48001122,PARENTING,The Christmas I'll Carry,None,"For me, it seems that to honor the holiday, I must honor the everyday.",the-christmas-ill-carry,"Jan 10, 2014"
175480de-21f8-11ec-bdf6-acde48001122,LIFESTYLE,Jessica Metzger Lost 110 Pounds On The Quest To Become Her Best Self,None,"I do not follow a specific meal plan, I do not eat 100 percent clean and I no longer track my calories.",i-lost-weight-jessica-metzger,"Jan 10, 2014"
17546f90-21f8-11ec-bdf6-acde48001122,PARENTING,6 Reasons Why Newborns Have Stuffy Noses,None,What many first-time parents don't realize is that newborns usually sound stuffy when they breathe.,why-newborns-have-stuffy-noses,"Jan 16, 2014"
175495ce-21f8-11ec-bdf6-acde48001122,LIFESTYLE,Unexpected Results From an Unexpected Resolution,None,"I usually don't go for New Year's resolutions, but I'm glad we did this one.",living-will,"Jan 17, 2014"
17544d9e-21f8-11ec-bdf6-acde48001122,LIFESTYLE,Time for Autism Speaks to Zip It?,None,The young father continued to talk about about his boy in a tone I found both desperate and loving.,time-for-autism-speaks-to,"Jan 17, 2014"
175442ae-21f8-11ec-bdf6-acde48001122,PARENTING,Flipping the Gridiron,Tackling My Son's Obsession With Football,"When I met my husband 14 years ago, he turned me into a football fan.",flipping-the-gridiron-tackling-my-sons-obsession-with-football,"Jan 17, 2014"
175485a2-21f8-11ec-bdf6-acde48001122,LIFESTYLE,Narcissists May Be More Likely To Be Leaders (But Not Necessarily Good Ones),None,"Indeed, a 2012 study Harms authored in the Journal of Applied Social Psychology showed that narcissists make better first.",narcissists-leaders,"Jan 21, 2014"
